In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import polars as pl
import seaborn as sns
from teeplot import teeplot as tp


In [ ]:
df = pl.read_parquet(
    "https://osf.io/gk2ty/download",
    use_pyarrow=True,
)


In [ ]:
print(df.columns)


In [ ]:
df = df.with_columns(
    pl.col("SLIP_INSERTION_BOOL_MASK").any().over(
        ["Treatment", "Run ID", "Generation Born"],
    ).alias("SLIP_INSERTION_BOOL_MASK any"),
)


In [ ]:
df = df.with_columns(
    pl.col("Is Task Coding Site").any().over(
        ["Treatment", "Run ID", "Generation Born", "Site"],
    )
    .alias("is any coding site"),
)


In [ ]:
df = df.with_columns(
    pl.col("Is Task Coding Site Delta").sum().over(
        ["Treatment", "Run ID", "Generation Born"],
    )
    .alias("is task coding site delta sum"),
)


In [ ]:
df = df.with_columns(
    pl.col("has task").sum().over(
        ["Treatment", "Run ID", "Generation Born", "Site"],
    )
    .alias("num tasks has"),
)


In [ ]:
df = df.with_columns(
    pl.col("is any coding site").sum().over(
        ["Treatment", "Run ID", "Generation Born", "Task",],
    ).alias("num coding sites"),
)


In [ ]:
dfx = df.group_by(
    ["Treatment", "Run ID", "Generation Born"],
).first()


In [ ]:
dfx = dfx.with_columns(
    (
        pl.col("num coding sites") / pl.col("Genome Length")
    ).alias("frac coding sites")
)


In [ ]:
tp.tee(
    sns.lineplot,
    hue="Treatment",
    y="num coding sites",
    x="Generation Born",
    data=dfx.to_pandas(),
    errorbar="se",
    teeplot_postprocess="plt.xscale('log')",
)


In [ ]:
tp.tee(
    sns.lineplot,
    hue="Treatment",
    y="frac coding sites",
    x="Generation Born",
    data=dfx.to_pandas(),
    errorbar="se",
    teeplot_postprocess="plt.xscale('log')",
)


In [ ]:
tp.tee(
    sns.lineplot,
    hue="Treatment",
    y="frac coding sites",
    x="num tasks has",
    data=dfx.to_pandas(),
    errorbar="se",
)


In [ ]:
tp.tee(
    sns.lineplot,
    hue="Treatment",
    y="num coding sites",
    x="num tasks has",
    data=dfx.to_pandas(),
    errorbar="se",
)
